# Importing Important Libraries and Creating S3 Bucket

In [1]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
bucket_name = 'ml-model-deployement' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
my_region = boto3.session.Session().region_name # set the region of the instance
print(my_region)

us-east-1


In [3]:
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [4]:
# set an output path where the trained model will be saved
prefix = 'xgboost-model'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://ml-model-deployement/xgboost-model/output


# Downloading The Dataset And Storing in S3

In [5]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [6]:
### Train Test split

import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [7]:
### Saving Train And Test Into Buckets
## We start with Train Data
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [8]:
# Test Data Into Buckets
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

### Building Models Xgboot- Inbuilt Algorithm

In [9]:
image_uri = sagemaker.image_uris.retrieve(
    framework='xgboost',
    region=my_region,
    version='1.7-1'
)

In [10]:
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [12]:
estimator = sagemaker.estimator.Estimator(image_uri=image_uri, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          use_spot_instances=True,
                                          max_run=300,
                                          max_wait=600)

In [13]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-07-05-15-05-49-145


2024-07-05 15:05:49 Starting - Starting the training job...
2024-07-05 15:06:04 Starting - Preparing the instances for training...
2024-07-05 15:06:36 Downloading - Downloading input data...
2024-07-05 15:07:07 Downloading - Downloading the training image......
2024-07-05 15:08:09 Training - Training image download completed. Training in progress.
2024-07-05 15:08:09 Uploading - Uploading generated training model[2024-07-05 15:08:04.857 ip-10-0-152-167.ec2.internal:8 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-07-05 15:08:04.878 ip-10-0-152-167.ec2.internal:8 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-07-05:15:08:05:INFO] Imported framework sagemaker_xgboost_container.training
[2024-07-05:15:08:05:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2024-07-05:15:08:05:INFO] No GPUs detected (normal if no gpus installed)
[2024-07-05:15:08:05:INFO] Running XGBoost Sagemaker in algorithm mode


### Deploy Machine Learning Model As Endpoints

In [14]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m5.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-07-05-15-09-14-080
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-07-05-15-09-14-080
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-07-05-15-09-14-080


------!

#### Prediction of the Test Data

In [22]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer

# Prepare the test data
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values
xgb_predictor.content_type = 'text/csv'

# Set the content type and serializer for the predictor
xgb_predictor.serializer = CSVSerializer()
xgb_predictor.deserializer = CSVDeserializer()
predictions = xgb_predictor.predict(test_data_array)

# Convert the predictions into an array
predictions_array = np.array(predictions, dtype=float)
predictions_array = predictions_array.T
print(predictions_array.shape)

(1, 12357)


In [23]:
predictions_array

array([[0.05214286, 0.05660191, 0.05096195, ..., 0.03436061, 0.02942475,
        0.03715819]])

In [24]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 



#### Deleting The Endpoints

In [26]:
xgb_predictor.delete_endpoint()
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': 'QNXAGMTJEFXJ1R62',
   'HostId': '43mkToKwSIlAOYjK1WSjZ+VRomuT8DF3jx61mv7ZKOFwRSaHxcYwcIWMJDOTtobVWrzvHAmZb3s=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': '43mkToKwSIlAOYjK1WSjZ+VRomuT8DF3jx61mv7ZKOFwRSaHxcYwcIWMJDOTtobVWrzvHAmZb3s=',
    'x-amz-request-id': 'QNXAGMTJEFXJ1R62',
    'date': 'Fri, 05 Jul 2024 15:20:20 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-model/output/sagemaker-xgboost-2024-07-05-15-05-49-145/debug-output/index/000000000/000000000030_worker_0.json'},
   {'Key': 'xgboost-model/output/sagemaker-xgboost-2024-07-05-15-05-49-145/debug-output/events/000000000030/000000000030_worker_0.tfevents'},
   {'Key': 'xgboost-model/output/sagemaker-xgboost-2024-07-05-15-05-49-145/debug-output/index/000000000/000000000010_worker_0.json'},
   {'Key': 'xgboost-model/output/sagemaker-xgboos